# Building Advanced Multi-Agent Systems with Tools, Handoffs, and Structured Outputs

Hey everyone! Welcome back to another exciting lesson on Agentic AI! 🎉

In this lab, we'll explore:

- **Agent Tools**: Using hosted tools (WebSearchTool), building custom function tools and using agents as tools
- **Agent Handoffs**: Implementing intelligent routing between specialist agents
- **Structured Outputs**: Using Pydantic models for deterministic, type-safe responses

Let's get started!

# Agent Tools

In [1]:
import os
from agents import Agent, Runner, WebSearchTool, function_tool
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### Hosted Tools

These are tools provided to us by OpenAI. They are plug and play which means they don't require us to write out functions or do much configuration.

In [9]:
financial_research_agent = Agent(
    name="FinancialResearchBot",
    model="gpt-4.1-mini",
    instructions="You are a financial research assistant. Provide detailed and accurate financial information about companies by looking up their financial reports."
)

In [6]:
financial_research_agent = Agent(
    name="FinancialResearchBot",
    model="gpt-4.1-mini",
    instructions="You are a financial research assistant. Provide detailed and accurate financial information about companies by looking up their financial reports.",
    tools=[WebSearchTool()]
)

In [8]:
results = await Runner.run(financial_research_agent, "I want you to make a recommendation on whether to buy, sell or hold Apple stock based on the current market conditions.")
print(results.final_output)

To provide a well-informed recommendation on whether to buy, sell, or hold Apple stock, I would need to review the latest financial data and market conditions for Apple Inc. (AAPL). Here’s the type of information I would analyze:

1. Recent Financial Performance:
   - Latest quarterly earnings and revenue
   - Profit margins and net income trends
   - Cash flow and balance sheet strength (cash reserves, debt levels)

2. Market Conditions:
   - Current stock price and valuation metrics (P/E ratio, PEG ratio)
   - Comparison to sector and broader market indices
   - Recent stock performance and volatility

3. Growth Prospects:
   - Product launches and innovation pipeline
   - Service segment growth (App Store, iCloud, etc.)
   - Market expansion opportunities

4. Analyst Ratings:
   - Consensus recommendations from Wall Street analysts
   - Target price vs current price

5. Risks:
   - Supply chain issues, regulation, or macroeconomic factors
   - Competitive landscape

Could you please

### Custom Tools

These are tools we build ourselves and provide our agents access to use them. Remember that all they are, are python functions that do a specific task (eg. call an API, get data from a database, perform a calculation/logic, etc.)

[Tavily](https://app.tavily.com/home) is a great API service which we will use for this example of trying to search the web for relevant results.

In [11]:
from tavily import TavilyClient
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

@function_tool
def tavily_search_tool(query: str, include_answer: str = "basic") -> str:
    """
    Search for information using the Tavily API.

    Args:
        query (str): The search query.
        include_answer (str): The level of detail to include in the answer. Options are "basic", "detailed", or "full".

    Returns:
        dict: The search results from Tavily.
    """
    client = TavilyClient(TAVILY_API_KEY)
    response = client.search(
        query=query,
        include_answer=include_answer
    )
    formatted_response = format_tavily_results(response)
    return formatted_response

In [12]:
tavily_search_tool.__dict__

{'name': 'tavily_search_tool',
 'description': 'Search for information using the Tavily API.',
 'params_json_schema': {'properties': {'query': {'description': 'The search query.',
    'title': 'Query',
    'type': 'string'},
   'include_answer': {'default': 'basic',
    'description': 'The level of detail to include in the answer. Options are "basic", "detailed", or "full".',
    'title': 'Include Answer',
    'type': 'string'}},
  'required': ['query', 'include_answer'],
  'title': 'tavily_search_tool_args',
  'type': 'object',
  'additionalProperties': False},
 'on_invoke_tool': <function agents.tool.function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool(ctx: 'ToolContext[Any]', input: 'str') -> 'Any'>,
 'strict_json_schema': True,
 'is_enabled': True,
 'tool_input_guardrails': None,
 'tool_output_guardrails': None}

In [10]:
def format_tavily_results(tavily_response: dict) -> str:
    """
    Format Tavily search results into a clean, LLM-friendly format.
    
    Args:
        tavily_response: Raw Tavily API response
        
    Returns:
        Formatted string with search results
    """
    formatted = f"# Search Results for: {tavily_response['query']}\n\n"
    
    # Add the AI-generated answer if available
    if tavily_response.get('answer'):
        formatted += f"## Summary\n{tavily_response['answer']}\n\n"
    
    # Add top search results
    formatted += "## Detailed Sources\n\n"
    for idx, result in enumerate(tavily_response['results'][:5], 1):  # Limit to top 5
        formatted += f"### {idx}. {result['title']}\n"
        formatted += f"**URL:** {result['url']}\n"
        formatted += f"**Relevance Score:** {result['score']:.2f}\n\n"
        formatted += f"{result['content'][:500]}...\n\n"  # Truncate long content
        formatted += "---\n\n"
    
    return formatted

In [13]:
tavily_search_tool

FunctionTool(name='tavily_search_tool', description='Search for information using the Tavily API.', params_json_schema={'properties': {'query': {'description': 'The search query.', 'title': 'Query', 'type': 'string'}, 'include_answer': {'default': 'basic', 'description': 'The level of detail to include in the answer. Options are "basic", "detailed", or "full".', 'title': 'Include Answer', 'type': 'string'}}, 'required': ['query', 'include_answer'], 'title': 'tavily_search_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000024768B84AE0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [14]:
vacation_agent = Agent(
    name="VacationPlannerBot",
    model="gpt-4.1-mini",
    instructions="You are a vacation planning assistant. Help users plan their vacations by providing recommendations on destinations, activities, and accommodations.",
    tools=[tavily_search_tool]
)

In [15]:
results = await Runner.run(vacation_agent, "I want to plan a vacation to Japan in the spring. Can you suggest some activities and places to visit?")
print(results)

RunResult:
- Last agent: Agent(name="VacationPlannerBot", ...)
- Final output (str):
    Visiting Japan in the spring is a wonderful choice, especially for enjoying the cherry blossoms and pleasant weather. Here are some great activities and places to consider for your trip:
    
    1. Tokyo:
       - Visit Ueno Park or Shinjuku Gyoen for beautiful cherry blossom viewing.
       - Explore the historic Asakusa district and Senso-ji Temple.
       - Enjoy shopping and entertainment in Shibuya and Harajuku.
    
    2. Kyoto:
       - See cherry blossoms at Maruyama Park and the Philosopher's Path.
       - Visit iconic temples such as Kinkaku-ji (Golden Pavilion) and Fushimi Inari Shrine.
       - Experience a traditional tea ceremony.
    
    3. Osaka:
       - Explore Osaka Castle and its surrounding park.
       - Visit the bustling Dotonbori area for street food and nightlife.
       - Take a day trip to nearby Nara to see the famous deer park and Todai-ji Temple.
    
    4. Hiros

In [16]:
results.final_output

"Visiting Japan in the spring is a wonderful choice, especially for enjoying the cherry blossoms and pleasant weather. Here are some great activities and places to consider for your trip:\n\n1. Tokyo:\n   - Visit Ueno Park or Shinjuku Gyoen for beautiful cherry blossom viewing.\n   - Explore the historic Asakusa district and Senso-ji Temple.\n   - Enjoy shopping and entertainment in Shibuya and Harajuku.\n\n2. Kyoto:\n   - See cherry blossoms at Maruyama Park and the Philosopher's Path.\n   - Visit iconic temples such as Kinkaku-ji (Golden Pavilion) and Fushimi Inari Shrine.\n   - Experience a traditional tea ceremony.\n\n3. Osaka:\n   - Explore Osaka Castle and its surrounding park.\n   - Visit the bustling Dotonbori area for street food and nightlife.\n   - Take a day trip to nearby Nara to see the famous deer park and Todai-ji Temple.\n\n4. Hiroshima:\n   - Visit the Peace Memorial Park and Museum.\n   - Take a ferry to Miyajima Island to see the famous floating torii gate.\n\n5. Ha

### Agents as Tools

We can provide our agents access to other agents as tools. This differs slightly from handoffs which we will have a look at later.

In [32]:
math_agent = Agent(
    name="MathSolverBot",
    model="gpt-4.1-mini",
    instructions="You are a math problem solver. Provide step-by-step solutions to mathematical problems.",
)

history_agent = Agent(
    name="HistoryExpertBot",
    model="gpt-4.1-mini",
    instructions="You are a history expert. Provide detailed historical information and context about various events and figures.",
)

triage_agent = Agent(
    name="TriageAgent",
    model="gpt-4.1-mini",
    instructions="You are a triage agent. Determine whether a user's query is best handled by the MathSolverBot or the HistoryExpertBot, and route the query accordingly.",
    tools=[
        math_agent.as_tool(
            tool_name="MathSolverTool",
            tool_description="Use this tool to solve mathematical problems."
        ),
        history_agent.as_tool(
            tool_name="HistoryExpertTool",
            tool_description="Use this tool to provide detailed historical information and context."
        )
        ]
)

In [33]:
results = await Runner.run(triage_agent, "What is the integral of x^2?")
print(results)

RunResult:
- Last agent: Agent(name="TriageAgent", ...)
- Final output (str):
    The integral of \( x^2 \) is \(\frac{x^3}{3} + C\), where \(C\) is the constant of integration.
- 3 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [34]:
results.__dict__

{'input': 'What is the integral of x^2?',
 'new_items': [ToolCallItem(agent=Agent(name='TriageAgent', handoff_description=None, tools=[FunctionTool(name='MathSolverTool', description='Use this tool to solve mathematical problems.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'MathSolverTool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002476C2D4720>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='HistoryExpertTool', description='Use this tool to provide detailed historical information and context.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'HistoryExpertTool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals

<div style="border-radius:16px;background:#1e2a1e;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
  <b style="color:#a3be8c;font-size:1.25em">Your Challenge:</b>
  <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6">
    <li><b>Build a News Research Agent</b> that can research current events on any topic and provide a comprehensive summary of recent articles.</li>
    <li>Choose your own approach: use <b>hosted tools</b> (like <code>WebSearchTool()</code>), <b>custom tools</b> (like Tavily API), or create an <b>agent routing system</b> with multiple specialist agents.</li>
    <li>Test your agent with at least 2 different news topics and see how well it summarizes the information!</li>
  </ul>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#a3be8c;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💪</div>
</div>

# Agent Handoffs

This is a different concept to agents as tools. Think of agents as tools as a way to supplement your main agent. Your main agent is able to make use of other tools and use the output that it gets back but the conversation with the user remains with the main agent.

In handoffs, you handoff the conversation to another agent (lets say a sales agent).

**Why might we do this?**
The use cases might be so niche that both these options will seem similar in terms of the actual output we get, the key difference is in the system instructions given to each agent. Each agent is going to have its own set of instructions and knowledge it has access to (as well as tone and style of writting for example) and this might be the deciding factor of wanting to use agents as tools or handoffs.

In [41]:
refund_agent = Agent(
    name="RefundAgent",
    model="gpt-4.1-mini",
    instructions="You are a refund agent that processes refund requests based on company policy. For now say there are no refunds available",
    handoff_description="Handles refund requests from customers."
)

retention_agent = Agent(
    name="RetentionAgent",
    model="gpt-4.1-mini",
    instructions="You are a retention agent that offers retention deals to customers considering cancellation. For now say that we are offering 20% off on everything",
    handoff_description="Handles customer retention and offers deals to prevent cancellations."
)

upsell_agent = Agent(
    name="SalesAgent",
    model="gpt-4.1-mini",
    instructions="You are a sales agent that offers upsell options to customers based on their current subscriptions. For now say we have a premium plan available at 30% off.",
    handoff_description="Handles upselling and cross-selling to customers."
)

customer_service_agent = Agent(
    name="CustomerServiceTriageAgent",
    model="gpt-4.1-mini",
    instructions="You are a customer service triage agent that chats with the user about their query then directs customer inquiries to the appropriate department based on the issue described. Do not ask for clarification.",
    handoffs=[refund_agent, retention_agent, upsell_agent]
)

In [42]:
results = await Runner.run(customer_service_agent, "I want to cancel my subscription and get a refund.")
print(results)

RunResult:
- Last agent: Agent(name="RefundAgent", ...)
- Final output (str):
    Currently, there are no refunds available according to our company policy. However, I can assist you with canceling your subscription if you would like.
- 5 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Here's how our customer service agent system works:

```mermaid
flowchart TD
    User([👤 User Input]) --> TriageAgent([🎯 Customer Service Triage Agent])
    
    TriageAgent -->|Refund Request| RefundAgent([💰 Refund Agent])
    TriageAgent -->|Cancellation/Retention| RetentionAgent([🤝 Retention Agent])
    TriageAgent -->|Upgrade/Upsell| UpsellAgent([📈 Sales Agent])
    
    RefundAgent --> Output1([📤 User Output])
    RetentionAgent --> Output2([📤 User Output])
    UpsellAgent --> Output3([📤 User Output])
    
    style TriageAgent fill:#d08770,stroke:#2e3440,stroke-width:3px,color:#2e3440
    style RefundAgent fill:#d08770,stroke:#2e3440,stroke-width:2px,color:#2e3440
    style RetentionAgent fill:#d08770,stroke:#2e3440,stroke-width:2px,color:#2e3440
    style UpsellAgent fill:#d08770,stroke:#2e3440,stroke-width:2px,color:#2e3440
    style User fill:#5e81ac,stroke:#2e3440,stroke-width:2px,color:#eceff4
    style Output1 fill:#5e81ac,stroke:#2e3440,stroke-width:2px,color:#eceff4
    style Output2 fill:#5e81ac,stroke:#2e3440,stroke-width:2px,color:#eceff4
    style Output3 fill:#5e81ac,stroke:#2e3440,stroke-width:2px,color:#eceff4
```

The **Customer Service Triage Agent** intelligently routes customer inquiries to the appropriate specialist agent based on the nature of the request. Each specialist agent then handles the query and provides a response back to the user.

# Structured outputs

Here's how our structured customer service workflow for the fictional company **Maven & Co.** operates:

```mermaid
flowchart TD
    Start([👤 User Input]) --> Classify([🔍 Classification Agent])
    
    Classify -->|books| BookAgent([📚 Book Agent])
    Classify -->|clothing| ClothingAgent([👗 Clothing Agent])
    Classify -->|retention| RetentionAgent([🎁 Retention Agent])
    Classify -->|order| OrderAgent([📦 Order Agent])
    
    BookAgent --> End1([📤 User Response])
    ClothingAgent --> End2([📤 User Response])
    RetentionAgent --> End3([📤 User Response])
    
    OrderAgent --> CheckApproval{Human Approval\nNeeded?}
    
    CheckApproval -->|No| End4([📤 User Response])
    CheckApproval -->|Yes| HumanReview([👤 Human Reviews Order])
    
    HumanReview -->|Approved| OrderSuccess([✅ Order Placed])
    HumanReview -->|Rejected| OrderCancel([❌ Order Cancelled])
    
    style Classify fill:#88c0d0,stroke:#2e3440,stroke-width:3px,color:#2e3440
    style BookAgent fill:#d08770,stroke:#2e3440,stroke-width:2px,color:#2e3440
    style ClothingAgent fill:#d08770,stroke:#2e3440,stroke-width:2px,color:#2e3440
    style RetentionAgent fill:#d08770,stroke:#2e3440,stroke-width:2px,color:#2e3440
    style OrderAgent fill:#d08770,stroke:#2e3440,stroke-width:2px,color:#2e3440
    style Start fill:#5e81ac,stroke:#2e3440,stroke-width:2px,color:#eceff4
    style End1 fill:#5e81ac,stroke:#2e3440,stroke-width:2px,color:#eceff4
    style End2 fill:#5e81ac,stroke:#2e3440,stroke-width:2px,color:#eceff4
    style End3 fill:#5e81ac,stroke:#2e3440,stroke-width:2px,color:#eceff4
    style End4 fill:#5e81ac,stroke:#2e3440,stroke-width:2px,color:#eceff4
    style CheckApproval fill:#ebcb8b,stroke:#2e3440,stroke-width:2px,color:#2e3440
    style HumanReview fill:#b48ead,stroke:#2e3440,stroke-width:2px,color:#eceff4
    style OrderSuccess fill:#a3be8c,stroke:#2e3440,stroke-width:2px,color:#2e3440
    style OrderCancel fill:#bf616a,stroke:#2e3440,stroke-width:2px,color:#eceff4
```

This workflow demonstrates a **deterministic, structured output system** where:
- The **Classification Agent** routes requests to specialized agents
- Each specialist agent returns structured data (Pydantic models)
- The **Order Agent** includes human-in-the-loop approval for critical actions
- All interactions end with a user response or final state

### Maven & Co. Agents

In [2]:
from pydantic import BaseModel, Field

class ClassificationResponse(BaseModel):
    category: str
    reasoning: str

In [ ]:
{
    'category': [],
    'reasoning': []
}

In [3]:
classification_agent = Agent(
    name="ClassificationAgent",
    model="gpt-4.1-mini",
    instructions="You are a classification agent that categorizes customer inquiries into one of four categories: books, clothing, retention, or order. Analyze the user's query and determine which category best fits their request.",
    output_type=ClassificationResponse
)

In [4]:
results = await Runner.run(classification_agent, "I want to return a book I ordered last week.")
print(results)

RunResult:
- Last agent: Agent(name="ClassificationAgent", ...)
- Final output (ClassificationResponse):
    {
      "category": "order",
      "reasoning": "The user's request involves returning a book they ordered, which pertains to the order process rather than the book's content or category itself. Therefore, the best fit is 'order' as it deals with transaction issues like returns."
    }
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [8]:
results.final_output.category

'order'

In [14]:
from pydantic import BaseModel, Field

class ClassificationResponse(BaseModel):
    reason: str = Field(..., description="The reasoning behind the classification")
    category: str = Field(..., description="The category of the customer inquiry")
    confidence: float = Field(..., description="The confidence level of the classification from 0 to 1")
    

class BookResponse(BaseModel):
    reason: str = Field(..., description="The reason for the book recommendation")
    response: str = Field(..., description="Personalized book recommendation message")


class ClothingResponse(BaseModel):
    reason: str = Field(..., description="The reason for the clothing recommendation")
    response: str = Field(..., description="Personalized clothing recommendation message")


class RetentionResponse(BaseModel):
    offer: str = Field(..., description="The retention offer being provided")
    discount_percentage: float = Field(..., description="Discount percentage offered")
    response: str = Field(..., description="Personalized retention message to the customer")


class OrderResponse(BaseModel):
    human_approval: bool = Field(..., description="Indicates if human approval is required for the order. Only to be used right before the order is about to be executed.")
    order_item_name: str = Field(..., description="Name of the item to be ordered")
    item_id: int = Field(..., description="Unique identifier of the item to be ordered")
    order_id: int = Field(..., description="Unique identifier for the order")
    order_amount: float = Field(..., description="Total amount for the order in USD")
    response: str = Field(..., description="Next steps for completing the order")

In [23]:
# Classification Agent
classification_agent = Agent(
    name="ClassificationAgent",
    model="gpt-4.1-mini",
    instructions="You are a classification agent that categorizes customer inquiries into one of four categories: books, clothing, retention, or order. Analyze the user's query and determine which category best fits their request.",
    output_type=ClassificationResponse
)

# Book Agent with fictional inventory
book_inventory = """
Available Books:
1. "The Midnight Library" by Matt Haig - $16.99 (12 in stock)
2. "Atomic Habits" by James Clear - $19.99 (25 in stock)
3. "The Psychology of Money" by Morgan Housel - $18.50 (8 in stock)
4. "Project Hail Mary" by Andy Weir - $21.99 (15 in stock)
5. "The Thursday Murder Club" by Richard Osman - $17.99 (20 in stock)
"""

book_agent = Agent(
    name="BookAgent",
    model="gpt-4.1-mini",
    instructions=f"You are a book specialist agent. Help customers find books from our inventory. {book_inventory}",
    output_type=BookResponse
)

# Clothing Agent with fictional inventory
clothing_inventory = """
Available Women's Clothing:
1. "Elegant Silk Blouse" - Size M - $89.99 (10 in stock)
2. "Classic Denim Jacket" - Size L - $124.99 (5 in stock)
3. "Floral Summer Dress" - Size S - $79.99 (18 in stock)
4. "Cashmere Cardigan" - Size M - $149.99 (7 in stock)
5. "High-Waist Trousers" - Size L - $94.99 (12 in stock)
"""

clothing_agent = Agent(
    name="ClothingAgent",
    model="gpt-4.1-mini",
    instructions=f"You are a women's clothing specialist agent. Help customers find clothing items from our inventory. {clothing_inventory}",
    output_type=ClothingResponse
)

# Retention Agent
retention_agent_structured = Agent(
    name="RetentionAgent",
    model="gpt-4.1-mini",
    instructions="You are a customer retention specialist. Your goal is to retain customers by offering them attractive promotional deals. Offer a 25% discount valid for 30 days on their next purchase. Be persuasive but respectful.",
    output_type=RetentionResponse
)

# Order Agent
order_agent = Agent(
    name="OrderAgent",
    model="gpt-4.1-mini",
    instructions="You are an order processing agent. Guide customers through placing their orders." \
    " Discuss items they want to purchase, quantities, shipping preferences, and payment options. Be helpful and thorough." \
    " Always ask for human approval before finalizing any order.",
    output_type=OrderResponse
)

### Workflow Implementation

In [24]:
input_list = []

In [25]:

while True:
    # Step 1: Get a user response
    user_input = input("You: ")
    print(f"User Input: {user_input}\n")
    input_list.append({'role': 'user', 'content': user_input})

    if user_input.lower() in ["exit", "quit"]:
        print("Exiting the system. Goodbye!")
        break

    # Step 2: Classify the user query
    print("Classifying the query...")
    classification_results = await Runner.run(classification_agent, input_list)
    classification_output = classification_results.final_output
    input_list = classification_results.to_input_list()
    print(f"Classification: {classification_output.category}\n (Confidence: {classification_output.confidence:.2f})")
    print(f"Reasoning: {classification_output.reason}\n")

    # Step 3: Route to appropriate agent based on classification
    if classification_output.category == "books":
        print("Routing to Book Agent...")
        book_results = await Runner.run(book_agent, input_list)
        book_output = book_results.final_output
        input_list = book_results.to_input_list()
        print("Book Response: ", book_results.final_output.response)
        print("Reason: ", book_results.final_output.reason)

    elif classification_output.category == "clothing":
        print("Routing to Clothing Agent...")
        clothing_results = await Runner.run(clothing_agent, input_list)
        clothing_output = clothing_results.final_output
        input_list = clothing_results.to_input_list()
        print("Clothing Response: ", clothing_results.final_output.response)
        print("Reason: ", clothing_results.final_output.reason)

    elif classification_output.category == "retention":
        print("Routing to Retention Agent...")
        retention_results = await Runner.run(retention_agent_structured, input_list)
        retention_output = retention_results.final_output
        input_list = retention_results.to_input_list()
        print("Retention Response: ", retention_results.final_output.response)
        print("Reason: ", retention_results.final_output.reason)

    elif classification_output.category == "order":
        print("Routing to Order Agent...")
        agent_results = await Runner.run(order_agent, input_list)
        order_output = agent_results.final_output
        input_list = agent_results.to_input_list()

        if order_output.human_approval:
            print("Order requires human approval. Escalating to a human agent.")
            print(f"  Item: {order_output.order_item_name}")
            print(f" Order ID: {order_output.order_id}")
            print(f" Amount: ${order_output.order_amount:.2f}")

            approval = input("Approve order? (yes/no): ")

            if approval.lower() == "yes":
                print("Order approved and processed.")
            else:
                print("Order denied.")
        else:
            print("Order processed automatically.")
            print(order_output.response)

    else:
        print("Unable to classify the inquiry. Please contact customer support for further assistance.")

User Input: I want a book similar to mockingjay

Classifying the query...
Classification: books
 (Confidence: 0.95)
Reasoning: The user is asking for a book similar to 'Mockingjay', which clearly relates to books rather than clothing, retention, or order issues.

Routing to Book Agent...
Book Response:  Based on your interest in 'Mockingjay', I recommend 'The Midnight Library' by Matt Haig. It offers a captivating exploration of choices and parallel lives with a thrilling and emotional story that fans of dystopian and speculative fiction often enjoy.
Reason:  Since you enjoyed 'Mockingjay', which is a thrilling dystopian novel with strong characters and suspense, I recommend 'The Midnight Library' by Matt Haig. It explores life choices and alternate realities with an engaging and thought-provoking narrative, appealing to readers who appreciate deep, compelling stories.
User Input: What clothing articles are similar to this theme of the mockingjay?

Classifying the query...
Classificati

<div style="border-radius:16px;background:#1e2a1e;margin:1em 0;padding:1em 1em 1em 3em;color:#eceff4;position:relative;box-shadow:0 6px 16px rgba(0,0,0,.4)">
  <b style="color:#a3be8c;font-size:1.25em">Your Challenge:</b>
  <ul style="margin:.6em 0 0;padding-left:1.2em;line-height:1.6">
    <li><b>Deploy Maven & Co. as a Web App!</b> Take the customer service workflow you just learned and turn it into an interactive application using either <b>Gradio</b> or <b>Streamlit</b>.</li>
    <li>Your app should:
      <ul style="margin-top:0.3em">
        <li>Accept user input through a chat interface</li>
        <li>Display the classification results (category and confidence)</li>
        <li>Show agent responses in a user-friendly format</li>
        <li>Handle the human approval step for orders interactively</li>
      </ul>
    </li>
  </ul>
  <div style="position:absolute;top:-.8em;left:-.8em;width:2.4em;height:2.4em;border-radius:50%;background:#a3be8c;color:#2e3440;display:flex;align-items:center;justify-content:center;font-weight:700;font-size:1.2em">💪</div>
</div>